In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
import seaborn as sns


In [3]:
df = pd.read_csv("data/recipes_w_search_terms.csv")
df_2 = pd.read_csv("data/recipes.csv")

In [4]:
df.head(1)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low..."


In [5]:
df_2.head(1)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."


## Merge recipe dataset with searchterms 

In [6]:
df_2 = pd.merge(df_2, df[['id', 'search_terms']], how= 'left', left_on = 'RecipeId', right_on='id')
df_2.head(1)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,id,search_terms
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...",38.0,"{'low-calorie', 'low-sodium', 'dessert', 'heal..."


In [7]:
df_2.drop('id', axis=1, inplace=True)

In [8]:
df_2.head(1)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,search_terms
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...","{'low-calorie', 'low-sodium', 'dessert', 'heal..."


In [9]:
df_searchterms = df_2[['RecipeId','search_terms']]

In [10]:
df_searchterms.head()

,RecipeId,search_terms
0,38,"{'low-calorie', 'low-sodium', 'dessert', 'heal..."
1,39,"{'indian', 'dinner', 'low-sodium'}"
2,40,"{'healthy', 'low-fat', 'low-sodium'}"
3,41,"{'side', 'low-calorie', 'vegetarian', 'low-carb'}"
4,42,"{'low-calorie', 'vegetarian', 'healthy', 'low-..."


In [11]:
import ast
df_searchterms['search_terms'] = df_searchterms['search_terms'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

# Flatten the sets into a single list
all_terms = [term for terms_set in df_searchterms['search_terms'] if pd.notna(terms_set) for term in terms_set]

# Create a pandas Series to easily get unique values and their counts
terms_series = pd.Series(all_terms)

# Get unique values and their counts
unique_terms_counts = terms_series.value_counts(dropna=False)

# Display the result
unique_terms_counts

/var/folders/83/jmlgpb4s3yl470xp9xbv89n40000gn/T/ipykernel_12868/1654578159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_searchterms['search_terms'] = df_searchterms['search_terms'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)


dinner         215831
dessert        116353
low-sodium      94177
low-carb        89648
low-calorie     76516
                ...  
no flour           20
no meat            11
carb-free           7
sodium-free         4
flour-less          3
Name: count, Length: 98, dtype: int64

In [12]:
unique_terms_counts.head(30) #unique_terms_counts.shape

dinner         215831
dessert        116353
low-sodium      94177
low-carb        89648
low-calorie     76516
healthy         70938
vegetarian      69036
side            52576
low-fat         48050
appetizer       47842
chicken         46439
lunch           45055
breakfast       31196
salad           29113
bread           27818
soup            21121
vegan           20401
cake            20037
italian         17066
snack           16816
southern        15844
mexican         15629
cookie          14918
diabetic        13453
pasta           12925
casserole       11947
pork            11861
pie             11848
gluten-free     11809
beef            10676
Name: count, dtype: int64

In [13]:
all_terms = [term for terms_list in df_searchterms['search_terms'] if pd.notna(terms_list) for term in terms_list]

# Extract unique searchterms
unique_terms = list(set(all_terms))
sorted_unique_terms = sorted(unique_terms)

# Display searchterms in alphabetic order
print(sorted_unique_terms)

['american', 'appetizer', 'atkins', 'baked', 'barbecue', 'beef', 'braised', 'brazilian', 'bread', 'breakfast', 'cajun', 'cake', 'carb-free', 'caribbean', 'casserole', 'chicken', 'chilean', 'chinese', 'cookie', 'creole', 'cuban', 'dairy-free', 'dessert', 'diabetic', 'diet', 'dinner', 'drink', 'egyptian', 'english', 'ethiopian', 'fat-free', 'fish', 'flour-less', 'flourless', 'french', 'german', 'gluten-free', 'grain-free', 'greek', 'healthy', 'indian', 'indonesian', 'irish', 'italian', 'jamaican', 'japanese', 'korean', 'lactose-free', 'lamb', 'lebanese', 'light', 'low-calorie', 'low-carb', 'low-fat', 'low-sodium', 'low-sugar', 'lowfat', 'lunch', 'malaysian', 'meatless', 'mexican', 'moroccan', 'nigerian', 'no flour', 'no meat', 'noodles', 'pasta', 'persian', 'peruvian', 'pie', 'pizza', 'polish', 'pork', 'pudding', 'quick', 'rice', 'roast', 'russian', 'salad', 'salt-free', 'sandwich', 'scottish', 'seafood', 'shrimp', 'side', 'snack', 'sodium-free', 'soup', 'southern', 'spanish', 'stew', 's

## Create diet specific columns

#### Create diet specifc lists based on searchterms


In [17]:
vegan_recipe_list = df_2[df_2['search_terms'].str.contains('vegan', na=False)]['RecipeId'].tolist()
vegetarian_recipe_list = df_2[df_2['search_terms'].str.contains('vegetarian', na=False)]['RecipeId'].tolist()
nomeat_recipe_list = df_2[df_2['search_terms'].str.contains('no meat', na=False)]['RecipeId'].tolist()

lactosefree_recipe_list = df_2[df_2['search_terms'].str.contains('lactose-free', na=False)]['RecipeId'].tolist()
dairyfree_recipe_list = df_2[df_2['search_terms'].str.contains('dairy-free', na=False)]['RecipeId'].tolist()

glutenfree_recipe_list = df_2[df_2['search_terms'].str.contains('gluten-free', na=False)]['RecipeId'].tolist()
grainfree_recipe_list = df_2[df_2['search_terms'].str.contains('grain-free', na=False)]['RecipeId'].tolist()
flour_less_recipe_list = df_2[df_2['search_terms'].str.contains('flour-less', na=False)]['RecipeId'].tolist()
flourless_recipe_list = df_2[df_2['search_terms'].str.contains('flourless', na=False)]['RecipeId'].tolist()
noflour_recipe_list = df_2[df_2['search_terms'].str.contains('no flour', na=False)]['RecipeId'].tolist()

#### Create diet specific columns 

In [ ]:
#Create vegan column
df_2['vegan'] = 0

#Update column based on RecipeIds in veganlist  
for recipe in vegan_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'vegan'] = 1


In [ ]:
df_2.vegan.value_counts()

In [ ]:
#Create vegetarian column
df_2['vegetarian'] = 0

#Update column based on RecipeIds in vegetarian-list  
for recipe in vegetarian_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'vegetarian'] = 1 

In [ ]:
df_2.vegetarian.value_counts()

In [ ]:
#Update column based on RecipeIds in nomeat-list  
for recipe in nomeat_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'vegetarian'] = 1 

In [ ]:
#Update column based on RecipeIds in vegan-list  
for recipe in vegan_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'vegetarian'] = 1

In [ ]:
df_2.vegetarian.value_counts()

In [16]:
#Create lactosefree column
df_2['lactosefree'] = 0 

#Update column based on RecipeIds in lactosefree- & dairyfree-lists  
for recipe in lactosefree_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'lactosefree'] = 1
for recipe in dairyfree_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'lactosefree'] = 1

NameError: name 'lactosefree_recipe_list' is not defined

In [ ]:
#Update column based on RecipeIds in vegan-list  
for recipe in vegan_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'lactosefree'] = 1

In [ ]:
#Create glutenfree column
df_2['glutenfree'] = 0

#Update column based on RecipeIds in relevant lists
for recipe in glutenfree_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'glutenfree'] = 1
for recipe in grainfree_recipe_list: 
    df_2.loc[df_2['RecipeId'] == recipe, 'glutenfree'] = 1
for recipe in flour_less_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'glutenfree'] = 1
for recipe in flourless_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'glutenfree'] = 1
for recipe in noflour_recipe_list:
    df_2.loc[df_2['RecipeId'] == recipe, 'glutenfree'] = 1

#### Checks & file creation



In [19]:
df_2.head(1)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,search_terms,lactosefree
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...","{'low-calorie', 'low-sodium', 'dessert', 'heal...",0


In [18]:
# Create pickle-file
df_2.to_pickle('df_2.pkl')

In [15]:
# Create lactosefree specific dataframe
df_lactosefree = df_2.loc[df_2['lactosefree'] == 1].copy()

KeyError: 'lactosefree'

In [ ]:
#Create csv
df_2.to_csv('df_with_diets.csv', index=False)  
